In [19]:
import requests             
from bs4 import BeautifulSoup 
import csv                  
import webbrowser
import io
import pandas as pd

def display(content, filename='output.html'):
    with open(filename, 'wb') as f:
        f.write(content)
        webbrowser.open(filename)

def get_soup(session, url, show=False):
    r = session.get(url)
    if show:
        display(r.content, 'temp.html')

    if r.status_code != 200: # not OK
        print('[get_soup] status code:', r.status_code)
    else:
        return BeautifulSoup(r.text, 'html.parser')
    
def post_soup(session, url, params, show=False):
    '''Read HTML from server and convert to Soup'''

    r = session.post(url, data=params)
    
    if show:
        display(r.content, 'temp.html')

    if r.status_code != 200: # not OK
        print('[post_soup] status code:', r.status_code)
    else:
        return BeautifulSoup(r.text, 'html.parser')
    
def scrape(url, lang='ALL'):

    # create session to keep all cookies (etc.) between requests
    session = requests.Session()

    session.headers.update({
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:57.0) Gecko/20100101 Firefox/57.0',
    })


    items = parse(session, url + '?filterLang=' + lang)

    return items

def parse(session, url):
    '''Get number of reviews and start getting subpages with reviews'''

    print('[parse] url:', url)

    soup = get_soup(session, url)

    if not soup:
        print('[parse] no soup:', url)
        return

    num_reviews = soup.find('span', class_='reviews_header_count').text # get text
    num_reviews = num_reviews[1:-1] 
    num_reviews = num_reviews.replace(',', '')
    num_reviews = int(num_reviews) # convert text into integer
    print('[parse] num_reviews ALL:', num_reviews)

    url_template = url.replace('.html', '-or{}.html')
    print('[parse] url_template:', url_template)

    items = []

    offset = 0

    while(True):
        subpage_url = url_template.format(offset)

        subpage_items = parse_reviews(session, subpage_url)
        if not subpage_items:
            break

        items += subpage_items
        
        print(len(items))
        if len(items) >= 150:
            break
        
        if len(subpage_items) < 5:
            break

        offset += 5

    return items

def get_reviews_ids(soup):

    items = soup.find_all('div', attrs={'data-reviewid': True})

    if items:
        reviews_ids = [x.attrs['data-reviewid'] for x in items][::2]
        print('[get_reviews_ids] data-reviewid:', reviews_ids)
        return reviews_ids
    
def get_more(session, reviews_ids):

    url = 'https://www.tripadvisor.com/OverlayWidgetAjax?Mode=EXPANDED_HOTEL_REVIEWS_RESP&metaReferer=Hotel_Review'

    payload = {
        'reviews': ','.join(reviews_ids), # ie. "577882734,577547902,577300887",
        #'contextChoice': 'DETAIL_HR', # ???
        'widgetChoice': 'EXPANDED_HOTEL_REVIEW_HSX', # ???
        'haveJses': 'earlyRequireDefine,amdearly,global_error,long_lived_global,apg-Hotel_Review,apg-Hotel_Review-in,bootstrap,desktop-rooms-guests-dust-en_US,responsive-calendar-templates-dust-en_US,taevents',
        'haveCsses': 'apg-Hotel_Review-in',
        'Action': 'install',
    }

    soup = post_soup(session, url, payload)

    return soup

def parse_reviews(session, url):
    '''Get all reviews from one page'''

    print('[parse_reviews] url:', url)

    soup =  get_soup(session, url)

    if not soup:
        print('[parse_reviews] no soup:', url)
        return

    hotel_name = soup.find('h1', id='HEADING').text

    reviews_ids = get_reviews_ids(soup)
    if not reviews_ids:
        return

    soup = get_more(session, reviews_ids)

    if not soup:
        print('[parse_reviews] no soup:', url)
        return

    items = []

    for idx, review in enumerate(soup.find_all('div', class_='reviewSelector')):

        badgets = review.find_all('span', class_='badgetext')
        if len(badgets) > 0:
            contributions = badgets[0].text
        else:
            contributions = '0'

        if len(badgets) > 1:
            helpful_vote = badgets[1].text
        else:
            helpful_vote = '0'
        user_loc = review.select_one('div.userLoc strong')
        if user_loc:
            user_loc = user_loc.text
        else:
            user_loc = ''
            
        bubble_rating = review.select_one('span.ui_bubble_rating')['class']
        bubble_rating = bubble_rating[1].split('_')[-1]
        
        print(bubble_rating)

        item = {
            'review_body': review.find('p', class_='partial_entry').text,
            'review_date': review.find('span', class_='ratingDate')['title'], # 'ratingDate' instead of 'relativeDate'
            'rating' : bubble_rating
        }

        items.append(item)
#         print(len(items))
#         if (len(items) >= 1000):
#             break
        print('\n--- review ---\n')
#         for key,val in item.items():
#             print(' ', key, ':', val)

    print()

    return items

def write_in_csv(items, filename='results.csv',
                  headers=['hotel name', 'review title', 'review body',
                           'review date', 'contributions', 'helpful vote',
                           'user name' , 'user location', 'rating'],
                  mode='w'):

    print('--- CSV ---')

    with io.open(filename, mode, encoding="utf-8") as csvfile:
        csv_file = csv.DictWriter(csvfile, headers)

        if mode == 'w':
            csv_file.writeheader()

        csv_file.writerows(items)
        
DB_COLUMN   = 'review_body'
DB_COLUMN1 = 'review_date'
DB_COLUMN2 = 'rating'
DB_COLUMN3 = 'hotel name'
DB_COLUMN4 = 'review title'

start_urls = list(pd.read_csv('hotel.csv')['urls'])

lang = 'en'

headers = [ 
    DB_COLUMN, 
    DB_COLUMN1,
    DB_COLUMN2,
    DB_COLUMN3,
    DB_COLUMN4
]



In [20]:
url_df = pd.read_csv('mid_tier_europe.csv')
urls = url_df['urls']
for url in urls:
#     url = 'https://www.tripadvisor.com/Hotel_Review-g35805-d90986-Reviews-Hilton_Chicago_Magnificent_Mile_Suites-Chicago_Illinois.html'

    # get all reviews for 'url' and 'lang'
    items = scrape(url, lang)

    if not items:
        print('No reviews')
    else:
        # write in CSV
        filename = url.split('Reviews-')[1][:-5] + '__' + lang
        print('filename:', filename)
        write_in_csv(items, filename + '.csv', headers, mode='w')

[parse] url: https://www.tripadvisor.com/Hotel_Review-g186337-d6002570-Reviews-DoubleTree_by_Hilton_Hotel_Liverpool-Liverpool_Merseyside_England.html#REVIEWS?filterLang=en
[parse] num_reviews ALL: 1163
[parse] url_template: https://www.tripadvisor.com/Hotel_Review-g186337-d6002570-Reviews-DoubleTree_by_Hilton_Hotel_Liverpool-Liverpool_Merseyside_England-or{}.html#REVIEWS?filterLang=en
[parse_reviews] url: https://www.tripadvisor.com/Hotel_Review-g186337-d6002570-Reviews-DoubleTree_by_Hilton_Hotel_Liverpool-Liverpool_Merseyside_England-or0.html#REVIEWS?filterLang=en
[get_reviews_ids] data-reviewid: ['648977142', '642708557', '648495159', '648468095', '648454633']
30

--- review ---

50

--- review ---

50

--- review ---

40

--- review ---

50

--- review ---


5
[parse_reviews] url: https://www.tripadvisor.com/Hotel_Review-g186337-d6002570-Reviews-DoubleTree_by_Hilton_Hotel_Liverpool-Liverpool_Merseyside_England-or5.html#REVIEWS?filterLang=en
[get_reviews_ids] data-reviewid: ['6483765

[get_reviews_ids] data-reviewid: ['614787887', '614466864', '614448725', '614207135', '613864298']
50

--- review ---

50

--- review ---

30

--- review ---

50

--- review ---

30

--- review ---


105
[parse_reviews] url: https://www.tripadvisor.com/Hotel_Review-g186337-d6002570-Reviews-DoubleTree_by_Hilton_Hotel_Liverpool-Liverpool_Merseyside_England-or105.html#REVIEWS?filterLang=en
[get_reviews_ids] data-reviewid: ['613558501', '613519978', '613156900', '611966936', '611522539']
40

--- review ---

50

--- review ---

50

--- review ---

30

--- review ---

50

--- review ---


110
[parse_reviews] url: https://www.tripadvisor.com/Hotel_Review-g186337-d6002570-Reviews-DoubleTree_by_Hilton_Hotel_Liverpool-Liverpool_Merseyside_England-or110.html#REVIEWS?filterLang=en
[get_reviews_ids] data-reviewid: ['609025580', '608598214', '607631377', '606487404', '606152810']
50

--- review ---

40

--- review ---

50

--- review ---

50

--- review ---

50

--- review ---


115
[parse_reviews] 

[get_reviews_ids] data-reviewid: ['633942438', '633651756', '633146106', '632780025', '632773103']
50

--- review ---

30

--- review ---

10

--- review ---

30

--- review ---

30

--- review ---


60
[parse_reviews] url: https://www.tripadvisor.com/Hotel_Review-g186233-d192828-Reviews-DoubleTree_by_Hilton_Chester-Chester_Cheshire_England-or60.html#REVIEWS?filterLang=en
[get_reviews_ids] data-reviewid: ['632741537', '632613358', '632537277', '631777976', '631601281']
50

--- review ---

50

--- review ---

40

--- review ---

50

--- review ---

50

--- review ---


65
[parse_reviews] url: https://www.tripadvisor.com/Hotel_Review-g186233-d192828-Reviews-DoubleTree_by_Hilton_Chester-Chester_Cheshire_England-or65.html#REVIEWS?filterLang=en
[get_reviews_ids] data-reviewid: ['631565547', '631143055', '630750352', '630646772', '630576180']
50

--- review ---

30

--- review ---

50

--- review ---

10

--- review ---

50

--- review ---


70
[parse_reviews] url: https://www.tripadvisor.co

[get_reviews_ids] data-reviewid: ['643890884', '643718162', '642842998', '642752522', '642249716']
10

--- review ---

50

--- review ---

50

--- review ---

50

--- review ---

50

--- review ---


15
[parse_reviews] url: https://www.tripadvisor.com/Hotel_Review-g186337-d192614-Reviews-Liverpool_Marriott_Hotel_City_Centre-Liverpool_Merseyside_England-or15.html#REVIEWS?filterLang=en
[get_reviews_ids] data-reviewid: ['642029066', '641091470', '640309452', '638941440', '638671162']
50

--- review ---

40

--- review ---

40

--- review ---

50

--- review ---

50

--- review ---


20
[parse_reviews] url: https://www.tripadvisor.com/Hotel_Review-g186337-d192614-Reviews-Liverpool_Marriott_Hotel_City_Centre-Liverpool_Merseyside_England-or20.html#REVIEWS?filterLang=en
[get_reviews_ids] data-reviewid: ['638597972', '638110519', '637280494', '637244222', '636877290']
50

--- review ---

40

--- review ---

50

--- review ---

50

--- review ---

50

--- review ---


25
[parse_reviews] url: ht

[get_reviews_ids] data-reviewid: ['580451149', '580357957', '579925371', '579900559', '578894780']
40

--- review ---

40

--- review ---

50

--- review ---

40

--- review ---

20

--- review ---


125
[parse_reviews] url: https://www.tripadvisor.com/Hotel_Review-g186337-d192614-Reviews-Liverpool_Marriott_Hotel_City_Centre-Liverpool_Merseyside_England-or125.html#REVIEWS?filterLang=en
[get_reviews_ids] data-reviewid: ['578678774', '578306015', '578224198', '577559538', '577515064']
40

--- review ---

50

--- review ---

50

--- review ---

50

--- review ---

40

--- review ---


130
[parse_reviews] url: https://www.tripadvisor.com/Hotel_Review-g186337-d192614-Reviews-Liverpool_Marriott_Hotel_City_Centre-Liverpool_Merseyside_England-or130.html#REVIEWS?filterLang=en
[get_reviews_ids] data-reviewid: ['576520810', '575911143', '575692806', '575279923', '574027350']
20

--- review ---

20

--- review ---

50

--- review ---

50

--- review ---

20

--- review ---


135
[parse_reviews] ur

...

[get_reviews_ids] data-reviewid: ['615063105', '614523140', '613854813', '613710784', '613417080']
40

--- review ---

50

--- review ---

40

--- review ---

50

--- review ---

50

--- review ---


80
[parse_reviews] url: https://www.tripadvisor.com/Hotel_Review-g551752-d530299-Reviews-DoubleTree_by_Hilton_Edinburgh_Queensferry_Crossing-North_Queensferry_Fife_Scotland-or80.html#REVIEWS?filterLang=en
[get_reviews_ids] data-reviewid: ['613231236', '612376803', '611443571', '610667347', '608908906']
50

--- review ---

40

--- review ---

40

--- review ---

40

--- review ---

50

--- review ---


85
[parse_reviews] url: https://www.tripadvisor.com/Hotel_Review-g551752-d530299-Reviews-DoubleTree_by_Hilton_Edinburgh_Queensferry_Crossing-North_Queensferry_Fife_Scotland-or85.html#REVIEWS?filterLang=en
[get_reviews_ids] data-reviewid: ['608160988', '607941003', '607550852', '606947124', '606835250']
50

--- review ---

40

--- review ---

50

--- review ---

50

--- review ---

50

--- rev

[get_reviews_ids] data-reviewid: ['634972061', '634208597', '633887616', '633839414', '633807351']
50

--- review ---

10

--- review ---

30

--- review ---

50

--- review ---

30

--- review ---


30
[parse_reviews] url: https://www.tripadvisor.com/Hotel_Review-g186525-d197779-Reviews-Edinburgh_Marriott_Hotel-Edinburgh_Scotland-or30.html#REVIEWS?filterLang=en
[get_reviews_ids] data-reviewid: ['633426284', '633274509', '632937992', '632608044', '632475108']
30

--- review ---

20

--- review ---

40

--- review ---

30

--- review ---

20

--- review ---


35
[parse_reviews] url: https://www.tripadvisor.com/Hotel_Review-g186525-d197779-Reviews-Edinburgh_Marriott_Hotel-Edinburgh_Scotland-or35.html#REVIEWS?filterLang=en
[get_reviews_ids] data-reviewid: ['632402019', '630902193', '630618143', '629949828', '629410578']
50

--- review ---

50

--- review ---

50

--- review ---

50

--- review ---

30

--- review ---


40
[parse_reviews] url: https://www.tripadvisor.com/Hotel_Review-g1865

[get_reviews_ids] data-reviewid: ['583670563', '583440983', '583354925', '582293001', '582286907']
30

--- review ---

40

--- review ---

50

--- review ---

10

--- review ---

50

--- review ---


145
[parse_reviews] url: https://www.tripadvisor.com/Hotel_Review-g186525-d197779-Reviews-Edinburgh_Marriott_Hotel-Edinburgh_Scotland-or145.html#REVIEWS?filterLang=en
[get_reviews_ids] data-reviewid: ['581843396', '579651110', '579219214', '578926942', '578450105']
40

--- review ---

40

--- review ---

50

--- review ---

50

--- review ---

30

--- review ---


150
filename: Edinburgh_Marriott_Hotel-Edinburgh_Scotland.html#RE__en
--- CSV ---
[parse] url: https://www.tripadvisor.com/Hotel_Review-g186525-d10604723-Reviews-Courtyard_by_Marriott_Edinburgh-Edinburgh_Scotland.html#REVIEWS?filterLang=en
[parse] num_reviews ALL: 580
[parse] url_template: https://www.tripadvisor.com/Hotel_Review-g186525-d10604723-Reviews-Courtyard_by_Marriott_Edinburgh-Edinburgh_Scotland-or{}.html#REVIEWS?filter

[get_reviews_ids] data-reviewid: ['614608330', '613551524', '613475571', '613236937', '613086273']
40

--- review ---

40

--- review ---

50

--- review ---

50

--- review ---

40

--- review ---


100
[parse_reviews] url: https://www.tripadvisor.com/Hotel_Review-g186525-d10604723-Reviews-Courtyard_by_Marriott_Edinburgh-Edinburgh_Scotland-or100.html#REVIEWS?filterLang=en
[get_reviews_ids] data-reviewid: ['612935616', '612795462', '612649552', '611659758', '610586184']
30

--- review ---

50

--- review ---

40

--- review ---

50

--- review ---

40

--- review ---


105
[parse_reviews] url: https://www.tripadvisor.com/Hotel_Review-g186525-d10604723-Reviews-Courtyard_by_Marriott_Edinburgh-Edinburgh_Scotland-or105.html#REVIEWS?filterLang=en
[get_reviews_ids] data-reviewid: ['610417422', '610066411', '609464148', '608856235', '608366799']
20

--- review ---

30

--- review ---

10

--- review ---

30

--- review ---

40

--- review ---


110
[parse_reviews] url: https://www.tripadvisor

[get_reviews_ids] data-reviewid: ['612453823', '612138757', '611557552', '611271751', '608972965']
40

--- review ---

30

--- review ---

40

--- review ---

40

--- review ---

20

--- review ---


55
[parse_reviews] url: https://www.tripadvisor.com/Hotel_Review-g186525-d2314424-Reviews-Residence_Inn_by_Marriott_Edinburgh-Edinburgh_Scotland-or55.html#REVIEWS?filterLang=en
[get_reviews_ids] data-reviewid: ['607794164', '607024307', '606849087', '606821057', '606348557']
50

--- review ---

40

--- review ---

10

--- review ---

10

--- review ---

40

--- review ---


60
[parse_reviews] url: https://www.tripadvisor.com/Hotel_Review-g186525-d2314424-Reviews-Residence_Inn_by_Marriott_Edinburgh-Edinburgh_Scotland-or60.html#REVIEWS?filterLang=en
[get_reviews_ids] data-reviewid: ['605681917', '604099491', '603441992', '603136795', '602984381']
40

--- review ---

10

--- review ---

50

--- review ---

30

--- review ---

20

--- review ---


65
[parse_reviews] url: https://www.tripadviso